In [1]:
# Embedding with nomic-embed-text

import os
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings

In [2]:
# --------------------
# Setup
# --------------------
os.environ.setdefault("no_proxy", "127.0.0.1,localhost")
os.environ.setdefault("HTTPX_NO_PROXY", "127.0.0.1,localhost")

persist_dir = "nomic-emb"
embeddings = OllamaEmbeddings(model="nomic-embed-text")

# Load documents only if we need to create embeddings
if not os.path.exists(persist_dir) or not os.listdir(persist_dir):
    print("⚡ No existing DB found. Creating new embeddings...")

    # Load and split text
    loader = TextLoader("facts.txt")
    docs = loader.load()
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=200,
        chunk_overlap=50
    )
    docs = text_splitter.split_documents(docs)

    # Create Chroma DB and persist
    db = Chroma.from_documents(
        documents=docs,
        embedding=embeddings,
        persist_directory=persist_dir
    )
else:
    print("✅ Found existing DB. Loading without re-embedding...")
    db = Chroma(
        persist_directory=persist_dir,
        embedding_function=embeddings
    )

✅ Found existing DB. Loading without re-embedding...


In [3]:
# --------------------
# Run a test query
# --------------------
query = "What is an interesting fact about the English language?"
results = db.similarity_search(query, k=3)

for i, result in enumerate(results, start=1):
    print(f"\nResult {i}:")
    print(result.page_content)


Result 1:
88. A group of owls is called a parliament.
89. The first TV commercial aired on July 1, 1941, for the Bulova Watch Company.
90. The modern Italian language originated from the region of Tuscany.

Result 2:
61. The first FIFA World Cup took place in Uruguay in 1930.
62. The Rubik's cube was patented by Hungarian inventor Ernő Rubik.
63. Karaoke means "empty orchestra" in Japanese.

Result 3:
1. "Dreamt" is the only English word that ends with the letters "mt."
2. An ostrich's eye is bigger than its brain.
3. Honey is the only natural food that is made without destroying any kind of life.
